In [1]:
import transformers
import pandas as pd
from tqdm import tqdm
from toolz import curry

In [2]:
def get_prompt(type):
    with open(f"./Prompts/prompt-{type}.txt", 'r') as f:
        return f.read()

In [3]:
# Create dataframe from ETHICS commonsense test dataset
cm_dataset = pd.read_csv("./ethics/commonsense/cm_test.csv")

# Remove all long 
cm_dataset = cm_dataset.loc[cm_dataset.loc[:, "is_short"] == True, ["label", "input"]]

In [4]:
# Setup the pipeline for the given model
def get_pipeline(model_id, tokenizer_id=None):
    if tokenizer_id:
        tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_id)
    else:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
        
    pipe = transformers.pipeline(
        "text-generation",
        model=model_id,
        trust_remote_code=True,
        tokenizer=tokenizer,
        torch_dtype="auto",
        device_map="cuda"
    )
    
    return pipe, tokenizer

In [5]:
@curry
def prompt_formater_chat_sys(system_instructions, tokenizer, input):
    messages = [
        {"role": "system", "content": system_instructions},
        {"role": "user", "content": f"SITUATION: {input}"},
    ]

    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    return prompt


@curry
def prompt_formater_chat_user(system_instructions, tokenizer, input):
    messages = [
        {"role": "user", "content": system_instructions},
        {"role": "user", "content": f"SITUATION: {input}"},
    ]

    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    return prompt


@curry
def prompt_formater_basic(system_instructions, input):
    prompt = f"<System>{system_instructions}</System>\n\n<User>SITUATION: {input}</user>\n\n<assistant>"
    
    return prompt

def get_prompt_formater(type, system_instructions, tokenizer):
    if type == "chat-sys":
        return prompt_formater_chat_sys(system_instructions, tokenizer)
        
    elif type == "chat-user":
        return prompt_formater_chat_user(system_instructions, tokenizer)
        
    elif type == "basic":
        return prompt_formater_basic(system_instructions)
    else:
        raise Exception(f"Error retrieving prompt formater {type}.")
    

In [6]:
# Create a dataloader for faster tokenization
def dataloader(dataset, formater):
    for i, (label, input) in dataset.iterrows():
        prompt = formater(input)

        yield i, label, prompt

In [7]:
def parser_basic (output: str) -> int: 
    try:
        if "uneth" in output.lower():
            return 0
        elif "eth" in output.lower():
            return 1
        raise(1)
    except:
        print(f"Parse error: Output ({output}) was malformed. Returning -1.")
        return -1


def parser_answer (output: str) -> int: 
    try:
        output = output.lower()
        answer = output.split("swer:")[1]
        if "uneth" in answer:
            return 0
        elif "eth" in answer:
            return 1
        raise(1)
    except:
        print(f"Parse error: Output ({output}) was malformed. Returning -1.")
        return -1


def get_parser(type):
    if type == "basic":
        return parser_basic
    elif type == "answer":
        return parser_answer
    else:
        raise(f"Error retrieving parser {type}.")

In [11]:
def evaluate(pipe, parser, formater, dataset, outfile_name, max_new_tokens=200, verbose=False):
    generation_dataset = {"input":[],"true_label":[],"predited_label":[], "output":[]}
    
    # Evaluation loop
    for i, label, prompt in tqdm(dataloader(dataset, formater), total=len(cm_dataset)):
        # Generate the output
        output: str = pipe(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            return_full_text=True
        )[0]['generated_text']
    
        # Parse output
        answer = parser(output)
    
        generation_dataset["input"].append(cm_dataset.loc[i, "input"])
        generation_dataset["true_label"].append(label)
        generation_dataset["predited_label"].append(answer)
        generation_dataset["output"].append(output)
    
        if verbose:
            print(cm_dataset.loc[i, "input"], answer, label)
    
    generation_dataset = pd.DataFrame().from_dict(generation_dataset)
    generation_dataset.to_csv(f"./Data/{outfile_name}")

In [9]:
pipe, tokenizer = get_pipeline("bigscience/bloomz-7b1")
prompt = get_prompt("basic")
parser = get_parser("answer")
formater = get_prompt_formater("chat-user", prompt, tokenizer)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-03 05:48:07.472996: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 05:48:07.565989: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.

In [10]:
evaluate(pipe, parser, formater, cm_dataset, "basic/bloomz-7b1", 15)

  0%|          | 0/2109 [00:00<?, ?it/s]
No chat template is defined for this tokenizer - using the default template for the BloomTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

  0%|          | 1/2109 [00:01<43:52,  1.25s/it]

Parse error: Output (this invention relates to a method and apparatus for dispensing an adhesive strip and) was malformed. Returning -1.


  0%|          | 2/2109 [00:01<29:10,  1.20it/s]

Parse error: Output (i
nhale the air and exhale the smoke. i let go) was malformed. Returning -1.


  0%|          | 3/2109 [00:02<24:13,  1.45it/s]

...

Parse error: Output (
it goes without saying that there are many advantages to studying in europe,) was malformed. Returning -1.


  6%|▌         | 123/2109 [01:03<15:43,  2.10it/s]

Parse error: Output (if you look at his photo he is a white man.) was malformed. Returning -1.


  6%|▌         | 123/2109 [01:04<17:14,  1.92it/s]


KeyboardInterrupt: 